# CantonMonths


First, prepare a btl table merging Famlie and Gemeinde Table




In [9]:
val famKan = df_familie
        //.where($"fam_end_dat" >= rangeBegin && $"fam_beg_dat" <= rangeEnd)
        .join(df_gemeinde
                //.select("gde_id", "gde_kt")
                , $"fam_ra_gde_id" === $"gde_id", join_type).drop("fam_ra_gde_id")
        .withColumnRenamed("gde_kt", "Kanton")
        // TODO: merge rows with subsequent periods in same canton
        //.select("fam_nr", "fam_beg_dat", "fam_end_dat", "Kanton")
famKan.where($"fam_nr" === 1000008 || $"fam_nr" === 1008659).show(false)

Now let's get the app data for RA family, month and canton

In [1]:
import ch.zzeekk.spark.temporalquery.TemporalQueryUtil._

val join_type = "left"

In [2]:
val df_familie = spark.table("default.stg_Familie")
val df_gemeinde = spark.table("default.stg_Gemeinde")

In [3]:
val df_duplicates_counts = df_familie.groupBy("fam_nr").count().filter($"count" >= 2 )
val df_duplicates = df_duplicates_counts.join(df_familie, Seq("fam_nr"))
df_duplicates.show()
df_familie.where($"fam_nr" === 1000008).show
df_familie.filter($"fam_nr" === 1008659).show()

+-------+-----+------+-------------------+-------------------+----------+----------+----------+----------+------------+------------+----------+-------------+-----------+-----------+-------------------+-----------+
| fam_nr|count|fam_id|        fam_beg_dat|        fam_end_dat|fam_fehler|fam_ges_id|fam_spr_id|fam_prt_id|fam_fkt_intv|fam_saf_intv|fam_gde_id|fam_ra_gde_id|fam_ext_ref|fam_mut_uid|        fam_mut_dat|fam_aktuell|
+-------+-----+------+-------------------+-------------------+----------+----------+----------+----------+------------+------------+----------+-------------+-----------+-----------+-------------------+-----------+
|1000008|    7|     6|1900-01-01 00:00:00|2004-08-08 00:00:00|         0|      1000|         D|         1|           3|           0|       248|          248|       null|          2|2015-09-30 09:31:00|          0|
|1000008|    7|     7|2004-08-09 00:00:00|2006-11-19 00:00:00|         0|      1000|         D|         1|           3|           0|       141| 

+------+-------+-------------------+-------------------+----------+----------+----------+----------+------------+------------+----------+-----------+-----------+-------------------+-----------+------+----------------+----------+------+----------+----------+------------------------+----------+-------------------+-------------------+----------------+-----------+-----------------------+
|fam_id|fam_nr |fam_beg_dat        |fam_end_dat        |fam_fehler|fam_ges_id|fam_spr_id|fam_prt_id|fam_fkt_intv|fam_saf_intv|fam_gde_id|fam_ext_ref|fam_mut_uid|fam_mut_dat        |fam_aktuell|gde_id|gde_name        |gde_bta_id|Kanton|gde_adr_id|gde_ina_id|gde_bem                 |gde_spr_id|gde_beg_dat        |gde_end_dat        |gde_folge_gde_id|gde_mut_uid|gde_mut_dat            |
+------+-------+-------------------+-------------------+----------+----------+----------+----------+------------+------------+----------+-----------+-----------+-------------------+-----------+------+----------------+---------

In [7]:
famKan.where()

In [5]:
import ch.zzeekk.spark.temporalquery.TemporalQueryUtil._
//import ch.zzeekk.spark.temporalquery.IntervalDef._
import ch.zzeekk.spark.temporalquery._
import java.time.temporal.ChronoUnit
import java.sql._
import spark.implicits._
implicit val ss = spark

val intervalDef = ClosedInterval(Timestamp.valueOf("0001-01-01 00:00:00"), Timestamp.valueOf("9999-12-31 00:00:00"), DiscreteTimeAxis(ChronoUnit.DAYS))

implicit val tqc = TemporalClosedIntervalQueryConfig( fromColName="fam_beg_dat", toColName="fam_end_dat", intervalDef = intervalDef)

val red = famKan.temporalCombine()
red.where($"fam_nr" === 1000008 || $"fam_nr" === 1008659).show

+-------+-------------------+-------------------+------+
| fam_nr|        fam_beg_dat|        fam_end_dat|Kanton|
+-------+-------------------+-------------------+------+
|1000008|1900-01-01 00:00:00|2050-01-01 00:00:00|    ZH|
|1008659|1900-01-01 00:00:00|2050-01-01 00:00:00|    ZH|
+-------+-------------------+-------------------+------+

